<a href="https://colab.research.google.com/github/cbadenes/curso-pln/blob/main/notebooks/04_Extensiones_LDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1)  Instalación y dependencias

In [1]:
#!pip install numpy==1.26.4 tomotopy==0.13.0 pandas==2.2.2
!pip install --upgrade tomotopy

import tomotopy as tp
import pandas as pd
import numpy as np
from datetime import datetime

#2) Labeled LDA

In [2]:
# Crear modelo LLDA
llda = tp.PLDAModel()

# Datos de ejemplo con sus etiquetas
documentos = [
    (['tecnología'], 'Los nuevos avances en inteligencia artificial permiten procesar datos más rápido'),
    (['ciencia'], 'Los científicos descubren un nuevo planeta similar a la Tierra'),
    (['medicina'], 'Un nuevo tratamiento promete curar enfermedades cardíacas'),
    (['tecnología', 'ciencia'], 'Los supercomputadores ayudan a simular fenómenos físicos complejos')
]

# Añadir documentos al modelo
for labels, texto in documentos:
    llda.add_doc(words=texto.split(), labels=labels)

# Entrenar modelo
llda.train(100)

# Mostrar las palabras más relevantes para cada etiqueta (label)
for i, label in enumerate(llda.topic_label_dict):
    print(f"## Tópico '{label}':")
    # Obtener palabras y sus pesos
    for palabra, peso in llda.get_topic_words(i, top_n=10):
        barra = "█" * int(peso * 50)  # Visualización del peso
        print(f"{palabra:<20} {peso:>6.4f}  {barra}")  # Alineación fija
    print()


## Tópico 'tecnología':
permiten             0.0892  ████
Los                  0.0892  ████
rápido               0.0892  ████
más                  0.0892  ████
datos                0.0892  ████
procesar             0.0892  ████
artificial           0.0892  ████
inteligencia         0.0892  ████
en                   0.0892  ████
avances              0.0892  ████

## Tópico 'ciencia':
a                    0.1097  █████
Los                  0.1097  █████
planeta              0.0551  ██
complejos            0.0551  ██
físicos              0.0551  ██
fenómenos            0.0551  ██
simular              0.0551  ██
ayudan               0.0551  ██
supercomputadores    0.0551  ██
Tierra               0.0551  ██

## Tópico 'medicina':
nuevo                0.1380  ██████
Un                   0.1380  ██████
tratamiento          0.1380  ██████
promete              0.1380  ██████
curar                0.1380  ██████
enfermedades         0.1380  ██████
cardíacas            0.1380  ██████
supercomputad

In [4]:
text = "La inteligencia artificial ayuda a los científicos a desarrollar nuevos medicamentos para curar enfermedades."
doc = llda.make_doc(text.split())
topicos = llda.infer(doc)[0]

# Mostrar distribución para cada tópico
for topico_idx, prob in enumerate(topicos):
    etiqueta = llda.topic_label_dict[topico_idx]
    barra = "▓" * int(prob * 50)
    print(f"{etiqueta}: {prob:.2%} {barra}")
    # Mostrar palabras más relevantes del tópico
    palabras = [word for word, _ in llda.get_topic_words(topico_idx, top_n=3)]
    print(f"Palabras clave: {', '.join(palabras)}\n")

tecnología: 57.07% ▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓
Palabras clave: permiten, Los, rápido

ciencia: 42.87% ▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓
Palabras clave: a, Los, planeta

medicina: 0.06% 
Palabras clave: nuevo, Un, tratamiento



#3) Dynamic Topic Model

In [ ]:
import tomotopy as tp
from datetime import datetime

# Crear modelo dinámico (2 tópicos, 3 periodos de tiempo)
dtm = tp.DTModel(k=2, t=3)  # k tópicos, 3 periodos de tiempo

# Datos de ejemplo a través del tiempo
documentos_tiempo = [
    # 2020
    (2020, 'La pandemia global afecta a millones de personas en todo el mundo'),
    (2020, 'Los científicos trabajan en el desarrollo de vacunas'),
    # 2021
    (2021, 'Las campañas de vacunación avanzan en diferentes países'),
    (2021, 'Nuevas variantes del virus preocupan a los expertos'),
    # 2022
    (2022, 'La población mundial alcanza altos niveles de inmunidad'),
    (2022, 'Los países comienzan a levantar restricciones sanitarias')
]

# Añadir documentos
for año, texto in documentos_tiempo:
    # Calcular el timestep (0 para 2020, 1 para 2021, 2 para 2022)
    timestep = año - 2020
    # Añadir documento con su timestep correspondiente
    dtm.add_doc(words=texto.split(), timepoint=timestep)

# Entrenar modelo
dtm.train(10)

# Mostrar evolución de tópicos
print("\n Evolución de tópicos:\n")
for topic in range(dtm.k):
    print(f"\n Tópico {topic + 1}:")
    for t in range(dtm.num_timepoints):
        año = 2020 + t
        print(f"\n {año}:")
        # Mostrar palabras y sus pesos
        for palabra, peso in dtm.get_topic_words(topic, timepoint=t, top_n=5):
            barra = "█" * int(peso * 50)
            print(f"{palabra}: {peso:.4f} {barra}")